Evolutionary dynamics of organised crime and terrorist networks in an network explicit configuration
---

Louis Weyland

In [3]:
# Importing all the modules
from IPython.display import IFrame
import logging
import os
import sys

import graph_tool.all as gt
import networkit as nk
import networkx as nx
import numpy as np

# Get directory first
path = os.getcwd()
par_dir = os.path.abspath(os.path.join(path, "../../"))
# Import own module
sys.path.append(par_dir)
if True:
    from network_utils.network_combiner import NetworkCombiner
    from network_utils.network_converter import NetworkConverter
    from network_utils.network_reader import NetworkReader


ModuleNotFoundError: No module named 'graph_tool'

In [2]:
IFrame('https://www.nature.com/articles/s41598-019-46141-8.pdf',width=800,height=450)